<a href="https://colab.research.google.com/github/ArpitaChatterjee/FaceRecognition-Attendance/blob/main/Attendance_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip  install dlib==19.18.0

In [2]:
!pip install face_recognition
import dlib
import face_recognition

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=f6e72156adb8fb0a5e275924e544799bdfff1fc65d95c6ada14b27266220e8ba
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [3]:
!pip install opencv-python
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os

In [7]:
path = '/content/drive/My Drive/Colab Notebooks/Face Recognition/ImagesAttendance'
images=[]
classNames=[]
mylist= os.listdir(path)
print(mylist)


['Bill Gates.jpg', 'Jack Ma.jpg', 'Elon Musk.jpg']


In [9]:
for cl in mylist:
  curImg = cv2.imread(f'{path}/{cl}')
  images.append(curImg)
  classNames.append(os.path.splitext(cl)[0])
print(classNames)

['Bill Gates', 'Jack Ma', 'Elon Musk']


In [10]:
def findEncodings(images):
  encodeList =[]
  for img in images:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encode = face_recognition.face_encodings(img)[0]
    encodeList.append(encode)
  return encodeList

encodelistknown = findEncodings(images)
print(len(encodelistknown))

3


In [11]:
print('encoding Complete')

encoding Complete


In [12]:
from datetime import datetime

In [13]:
#TO Mark Attendence
def markAttendence(name):
  with open('/content/drive/My Drive/Colab Notebooks/Face Recognition/Attendance.csv', 'r+') as f:
    mydatalist = f.readlines()
    namelist=[]
    for line in mydatalist:
      entry = line.split(',')
      namelist.append(entry[0])
    if name not in namelist:
      now = datetime.now()
      dtstr = now.strftime('%H:%M:%S')
      f.writelines(f'\n{name}, {dtstr}')

markAttendence('Elon')
markAttendence('Bill Gates')

In [14]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [16]:
from IPython.display import Image
try:
  cap = take_photo()
  print('Saved to {}'.format(cap))
  
  # Show the image which was just taken.
  #display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

Saved to photo.jpg


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
#cap = cv2.VideoCapture(0)

while True:
  sucess, img = cap.read()
  imgS = cv2.resize(img, (0,0), None, 0.25, 0.25)
  imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

  facesCurFrame = face_recognition.face_locations(imgS)
  encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

  for encodeFace, faceloc in zip(encodesCurFrame, facesCurFrame):
    matches = face_recognition.compare_faces(encodelistknown, encodeFace)
    faceDis = face_recognition.face_distance(encodelistknown, encodeFace)
    print(faceDis)
    matchIndex = np.argmin(faceDis)

    if matches[matchIndex]:
      name=classNames[matchIndex].upper()
      print(name)
      y1,y1,y2,x1 =faceloc
      y1,y1,y2,x1 = y1*4, y1*4, y2*4, x1*4
      cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
      cv2.rectangle(img, (x1,y2-35), (x2,y2), (0,255,0), cv2.FILLED)
      cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)

      #when the face matches it will call the attendence fn and put the naem of the person detected by this 
      #mth directly to the Attendence.csv file that was created in order to mark attendence directly from 
      #face-recognition
      markAttendence(name)

cv2.imshow('webcam', img)
cv2.waitKey(0)

      


AttributeError: ignored